# Processamento dos dados
---


## Imports and constants

In [51]:
from PIL import Image
import json
import os
import pandas as pd
import numpy as np

from tqdm import tqdm


In [92]:
IMAGES_PATH = "subsample"
ANNOTATIONS_PATH = "annotations"
CROPPED_PATH = "cropped"

In [84]:
class_names = [
    'call',
    'dislike',
    'fist',
    'four',
    'like',
    'mute',
    'ok',
    'one',
    'palm',
    'peace',
    'rock',
    'stop',
    'stop_inverted',
    'three',
    'two_up',
    'two_up_inverted',
    'three2',
    'peace_inverted',
]

## Functions

In [17]:
def read_annotations(path):
        json_annotations = json.load(open(path))
        json_annotations = [dict(annotation, **{'name': f'{name}.jpg'}) for name, annotation in zip(json_annotations, json_annotations.values())]
        annotations = pd.DataFrame(json_annotations)
        
        labels = list(annotations['labels'])
        targets = []
        for label in labels:
            targets.append([item for item in label if item != 'no_gesture'][0])

        annotations['target'] = targets
        return annotations

In [47]:
def get_crop_from_bbox(image, bbox, box_scale=1.):
    
        width, height = image.size
        x1, y1, w, h = bbox
        bbox = [x1 * width, y1 * height, (x1 + w) * width, (y1 + h) * height]
        
        int_bbox = np.array(bbox).round().astype(np.int32)

        x1 = int_bbox[0]
        y1 = int_bbox[1]
        x2 = int_bbox[2]
        y2 = int_bbox[3]
        cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

        w = h = max(x2 - x1, y2 - y1)
        x1 = max(0, cx - box_scale * w // 2)
        y1 = max(0, cy - box_scale * h // 2)
        x2 = cx + box_scale * w // 2
        y2 = cy + box_scale * h // 2
        x1, y1, x2, y2 = list(map(int, (x1, y1, x2, y2)))

        crop_image = image.crop((x1, y1, x2, y2))
        bbox_orig = np.array([x1, y1, x2, y2]).reshape(2, 2)

        return crop_image, bbox_orig

## Example of dataset

In [22]:
df = read_annotations("annotations/call.json")
df.head()

,bboxes,labels,leading_hand,leading_conf,user_id,name,target
0,"[[0.42538839, 0.21308209, 0.05382926, 0.112731...",[call],right,1.0,4c2d70803e35ce2d30f3e875f1b01c6c,01898f3e-8422-4e6a-a056-30206f905640.jpg,call
1,"[[0.40016984, 0.3223251, 0.13044141, 0.12870407]]",[call],right,1.0,36d33267899bc102184bb6376896e1d1,0516ab39-9dd3-41bf-9707-ccea0dbf985f.jpg,call
2,"[[0.48937548, 0.56126298, 0.1826894, 0.07676647]]",[call],left,0.8,2584891344198940dc84b95e95e01730,055cbe75-f563-49e2-9c4f-86a1fb47b40f.jpg,call
3,"[[0.56409572, 0.19575476, 0.09491629, 0.119821...","[call, no_gesture]",right,1.0,c89784925167c2a730e910424e267534,082b2083-c056-4735-a9f6-3df3593283d9.jpg,call
4,"[[0.27973014, 0.3370414, 0.20686737, 0.14207432]]",[call],right,1.0,9944ad417ddf5c5730ced9b5cf302da5,09732089-243c-47e6-a314-264fae24871e.jpg,call


In [44]:
bbox = df.iloc[0]['bboxes'][0]
bbox

[0.42538839, 0.21308209, 0.05382926, 0.11273142]

## Processing images

In [93]:
for class_name in class_names:
    os.makedirs(f"{CROPPED_PATH}/{class_name}")

In [94]:
def process_class_images(class_name):
    df = read_annotations(f"{ANNOTATIONS_PATH}/{class_name}.json")
    
    with tqdm(total=df.shape[0]) as pbar:
        pbar.set_description(f"Processing {class_name}:")
        for row in df.iterrows():
            pbar.update(1)
            real_bbox = [row[1]["bboxes"][i] for i, value in enumerate(row[1]["labels"]) if value == class_name]
            img = Image.open(f"{IMAGES_PATH}/{class_name}/{row[1]['name']}")
            tmp_img, bbox = get_crop_from_bbox(img, real_bbox[0])
            tmp_img.save(f"{CROPPED_PATH}/{class_name}/{row[1]['name']}")

In [95]:
for class_name in class_names:
    process_class_images(class_name)

Processing peace_inverted:: 100%|█████████████| 100/100 [00:05<00:00, 18.39it/s]
